In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(os.getenv('PYTHONPATH')) 
import numpy as np
import matplotlib.pyplot as plt
import hcp_utils as hcp
import matplotlib.pyplot as plt
from nilearn import plotting
from matplotlib.colors import ListedColormap

#local
from src.utils.transforms import SelectROIs

In [ ]:
root = os.path.join(os.getenv("DATASETS_ROOT"), "MOSAIC")
project_root = os.path.join(os.getenv("PROJECT_ROOT"))
print(f"root: {root}")
print(f"project_root: {project_root}")
save_root = os.path.join(project_root, "src", "fmriDatasetPreparation", "visualizations", "output")
if not os.path.exists(save_root):
    os.makedirs(save_root)

In [ ]:
rois = [f"GlasserGroup_{x}" for x in range(1,23)] #["GlasserGroup_1", "GlasserGroup_2", "GlasserGroup_3","GlasserGroup_4", "GlasserGroup_5"] #["LO1","LO2"] #["V1"]
ROI_selection = SelectROIs(selected_rois=rois, remove_nan_vertices=False)

In [ ]:
            
ext_list = ['png'] #,'svg'] #only matters if save_flag is True 
save_flag=False

##label each vertex according to the one of 22 glasser groups it belongs to
stat_groups = [0] * 91282 
for vertex_id, group_name in ROI_selection.index_to_group.items():
    stat_groups[vertex_id] = int(group_name.split('_')[-1]) #just get the group number 1-22

#label each vertex according its roi index 1-180
stat_rois = hcp.mmp.map_all

#make colormaps
colors_groups = ((167/255, 167/255, 167/255),) + plt.colormaps['Set3'].colors + plt.colormaps['tab10'].colors #23 total colors
cmap_groups = ListedColormap(colors_groups)

#get group level colormap for left hemishphere
cortex_data_groups_left = hcp.left_cortex_data(stat_groups)
cortex_data_groups_right = hcp.right_cortex_data(stat_groups)

ids_left = np.unique(cortex_data_groups_left).astype(int)
ids_right = np.unique(cortex_data_groups_right).astype(int)

rgba_groups_left = np.zeros((len(ids_left), 4))
for i in range(len(ids_left)):
    ind = cortex_data_groups_left==ids_left[i]
    rgba_groups_left[i,:] = colors_groups[ids_left[i]] + (1,) #adds an alpha channel

rgba_groups_right = np.zeros((len(ids_right), 4))
for i in range(len(ids_right)):
    ind = cortex_data_groups_right==ids_right[i]
    rgba_groups_right[i,:] = colors_groups[ids_right[i]] + (1,) #adds an alpha channel

cortex_data_rois_left = hcp.left_cortex_data(stat_rois)
cortex_data_rois_right = hcp.right_cortex_data(stat_rois)

nrois_left = len(set(cortex_data_rois_left))
nrois_right = len(set(cortex_data_rois_right))

#create a figure with multiple axes to plot each anatomical image
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), subplot_kw={'projection': '3d'})
plt.subplots_adjust(wspace=0)
im = plotting.plot_surf_roi(hcp.mesh.flat_left, cortex_data_groups_left, symmetric_cmap=False, cmap=cmap_groups,axes=axes[0])
#add the left hemi contours
plotting.plot_surf_contours(hcp.mesh.flat_left, cortex_data_rois_left, colors=[(0,0,0,1)]*nrois_left,axes=axes[0])

im = plotting.plot_surf_roi(hcp.mesh.flat_right, cortex_data_groups_right, symmetric_cmap=False, cmap=cmap_groups,axes=axes[1])
#add the right hemi contours
#plotting.plot_surf_contours(hcp.mesh.flat_right, cortex_data_rois_right, colors=[(0,0,0,1)]*nrois_right,axes=axes[1])

#flip along the horizontal
axes[0].invert_yaxis()
axes[1].invert_yaxis()

if save_flag:
    for ext in ext_list:
        if ext == 'png':
            plt.savefig(os.path.join(save_root, f"fsLR32k_MMP_parcellation_mesh-flat.{ext}"),dpi=300)
        else:
            plt.savefig(os.path.join(save_root, f"fsLR32k_MMP_parcellation_mesh-flat.{ext}"))
plt.show()